In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
graph = Neo4jGraph()

/tmp/ipykernel_152858/267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [4]:
loader = TextLoader(file_path="dum.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [5]:
llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(llm=llm)

/tmp/ipykernel_152858/329755349.py:1: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")


In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [6]:
for relationship in graph_documents[0].relationships:
    print(relationship)

In [7]:
for node in graph_documents[0].nodes:
    print(node)

id='Rivershade' type='Location' properties={}
id='The Tides Of Rivershade' type='Book/story' properties={}


In [8]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Rivershade', type='Location', properties={}), Node(id='The Tides Of Rivershade', type='Book/story', properties={})], relationships=[], source=Document(metadata={'source': 'dum.txt'}, page_content='"The Tides of Rivershade"\n\nIn the town of Rivershade, nestled by the edge of a vast, mist-covered river, lives a multitude of characters, each unaware of how deeply their lives are intertwined until fate conspires to bring them together.'))

In [9]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [6]:
embeddings = OllamaEmbeddings(
    model="llama3.1",
)

In [7]:

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever(search_kwargs={"k": 10})

In [30]:
# Now we initialize from existing graph
existing_graph = Neo4jVector.from_existing_graph(
    embedding= embeddings,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)
result = existing_graph.similarity_search("who sophia?")


In [31]:
print([el.page_content for el in result])

['\ntext: founders and an ancient power that lived in the water.', '\ntext: is marked with no date or sender. Curious, Eleanor reads the contents:', '\ntext: was believed to have disappeared under mysterious circumstances—just like those who had been vanishing along the river recently.', '\ntext: her father had started before his mysterious death years ago.']


In [ ]:
result = vector_retriever.invoke("who are the characters in the story??")
print([el.page_content for el in result])

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

['\ntext: Chapter 1: The Forgotten Letter', '\ntext: is marked with no date or sender. Curious, Eleanor reads the contents:', '\ntext: founders and an ancient power that lived in the water.', '\ntext: Chapter 2: The Sinking Bridge', '\ntext: represents a part of the town’s history—landmarks that must be found and understood to restore balance between the river and the people of Rivershade.', '\ntext: the river better than anyone, and he is certain something is wrong.', '\ntext: her father had started before his mysterious death years ago.', '\ntext: "The truth of Rivershade’s founding is hidden in plain sight. The key is the river—look there."', '\ntext: recognizes from an old legend his grandmother used to tell him. The bridge, it turns out, is not just a structure, but a key element in a much older mystery.', '\ntext: these stories as superstition, begins to reconsider them after hearing Graham’s strange experiences.']


In [8]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [9]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [22]:
# entity_chain.invoke("What is the significance of the river in the town’s founding, and why is it tied to both the prosperity and the dangers of Rivershade?")

In [10]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [24]:
# print(graph_retriever("What is the significance of the river in the town’s founding, and why is it tied to both the prosperity and the dangers of Rivershade?"))

In [ ]:
#test vector store
db = Neo4jVector.from_documents(
    documents, embeddings, url= os.environ["NEO4J_URI"],username=os.environ["NEO4J_USERNAME"], password= os.environ["NEO4J_PASSWORD"]
)


In [ ]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    # docs_with_score = vector_index.similarity_search_with_score(question, k=2)
    # docs_with_score = db.similarity_search_with_score(question, k=2)
    print(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    print(vector_data)
    # print(docs_with_score)
    print(graph_data)
    # print("docs data = ",docs_with_score)


    final_data = f"""Relationships:
{graph_data}
facts:
{"#Document ". join(vector_data)}
    """
    return final_data

In [13]:
template = """
Answer the question based only on the following context:
{context}

You are an advanced AI designed to analyze and synthesize information from a single provided file: {context} containing both factual details and relationship connections about a subject. 
When answering a question, do not simply extract text but instead interpret and expand upon the provided information by logically inferring connections and implications. 
Identify key facts, analyze relationships, and generate well-structured responses that go beyond surface-level details while maintaining accuracy and coherence. 
Use contextual reasoning to provide insightful and relevant answers. If the required information is not found, acknowledge the limitation while avoiding speculation. 
Always maintain a neutral, well-supported, and logically sound tone in your responses.

Question: {question}
Use natural human language 


Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
chain.invoke(input="who is graham?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


who is graham?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

['\ntext: Chapter 1: The Forgotten Letter', '\ntext: founders and an ancient power that lived in the water.', '\ntext: is marked with no date or sender. Curious, Eleanor reads the contents:', '\ntext: Chapter 2: The Sinking Bridge', '\ntext: was believed to have disappeared under mysterious circumstances—just like those who had been vanishing along the river recently.', '\ntext: her father had started before his mysterious death years ago.', '\ntext: the river better than anyone, and he is certain something is wrong.', '\ntext: represents a part of the town’s history—landmarks that must be found and understood to restore balance between the river and the people of Rivershade.', '\ntext: these stories as superstition, begins to reconsider them after hearing Graham’s strange experiences.', '\ntext: This power, once a protector, has become restless and vengeful. Those who have disappeared? They were not lost—they were taken by the river. But the key to stopping it lies in the symbols Decl

"Graham appears to be an individual with some sort of connection to the mysterious experiences and events unfolding in Rivershade. He has shared his own strange experiences, which have caused Eleanor to reconsider her views on the town's stories as mere superstition. Graham's involvement suggests that he may possess knowledge or insight into the supernatural occurrences happening along the river."

In [ ]:
def showSgraph():
    driver = GraphDatabase.driver(
        uri= os.environ["NEO4J_URI"],
        auth= (os.environ["NEO4J_USERNAME"],
               os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    widget=GraphWidget(graph = session.run("MATCH p=()-[]->() RETURN p LIMIT 350;").graph())
    widget.node_label_mapping = 'id'
    return widget

showSgraph()

GraphWidget(layout=Layout(height='800px', width='100%'))